In [1]:
import pandas as pd
import math, json
import numpy as np
import datetime

In [2]:
df = pd.read_csv("standard_nov21frl.csv", sep=';')

In [3]:
def correct_flag_and_title(x):
    if x[12] in ['wi', 'i']:
        x[12] = 'i'
    else:
        x[12] = 'a'
    try:
        if math.isnan(x[4]):
            x[4] = 'R.A.S'
    except:
        pass
    return x

In [4]:
df_copy = df.copy()
df_copy = df_copy.apply(correct_flag_and_title, axis=1)

In [5]:
def replace_values(x):
    if x[0] == 'F':
        x[0] = 'Femme'
    else:
        x[0] = 'Homme'
    if x[2] == 'i':
        x[2] = 'Inactif'
    else:
        x[2] = 'Actif'
    return x

In [6]:
df_copy['eff'] = 1
final_df = df_copy.groupby(['sex', 'title', 'flag', 'country'], as_index=False).agg({'eff': 'sum'})
final_df = final_df.apply(replace_values, axis=1)
final_df = final_df[final_df.title!='R.A.S']
final_df

,sex,title,flag,country,eff
0,Femme,CM,Actif,COL,1
1,Femme,CM,Actif,CRC,1
2,Femme,CM,Actif,ESA,1
3,Femme,CM,Actif,FRA,1
4,Femme,CM,Actif,IND,1
...,...,...,...,...,...
2061,Homme,IM,Inactif,YEM,4
2062,Homme,IM,Inactif,ZAM,1
2063,Homme,IM,Inactif,ZIM,3
2434,Homme,WCM,Actif,EGY,1


_**For repartition\_3.json**_

In [14]:
final_data = {
    "name": "Joeurs",
    "children": []
}
for title in final_df.title.unique():
    title_json = {"name": title, "children":[]}
    tmp_title_df = final_df[final_df.title == title]
    for sex in tmp_title_df.sex.unique():
        sex_json = {"name": sex, "children":[]}
        tmp_sex_df = tmp_title_df[tmp_title_df.sex == sex]
        for country in tmp_sex_df.country.unique():
            country_json = {"name": country, "children":[]}
            tmp_country_df = tmp_sex_df[tmp_sex_df.country == country]
            for flag in tmp_country_df.flag.unique():
                flag_json = {"name": flag, 'value': tmp_country_df[tmp_country_df.flag == flag].eff.sum()}
                country_json["children"].append(flag_json)
            sex_json["children"].append(country_json)
        title_json["children"].append(sex_json)
    final_data["children"].append(title_json)

In [19]:
def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()
with open('standard_repartition.json', 'w') as outfile:
    json.dump(final_data, outfile, default=myconverter)